In [1]:
import os
import json

import opensim as osim
import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt


# Notes: 
- `R_HJC` & `L_HJC` don't exist in the `lenhart2015.osim` model. They were removed from the `marker_weights.json`
    - `R.Scapula` & `L.Scapula`
    - `R.Th4`
    - `L.SH4`
- Created a new external loads file in this directory that correctly points to the path of the GRF data: 
    `comak.set_external_loads_file('./overground_17_ext_loads.xml')`

In [2]:
matlab_path = os.path.abspath('../../../../matlab/')
models_path = os.path.abspath('../../../../models')

# model_file = os.path.join(models_path, 'knee_healthy', 'lenhart2015', 'lenhart2015.osim')

model_file = "/root/jam-resources/models/knee_healthy/lenhart2015/lenhart2015.osim";
coordinates_file = "/root/jam-resources/python/examples/Walking/Walking/results/comak-inverse-kinematics/overground_17_ik.mot";
external_loads_file = "/root/jam-resources/python/examples/Walking/Walking/overground_17_ext_loads.xml";
forceset_file =  "/root/jam-resources/models/knee_healthy/lenhart2015/lenhart2015_reserve_actuators.xml";
comak_result_dir = "/root/jam-resources/python/examples/Walking/Walking/results/comak";
results_basename = "walking";
save_xml_path = "/root/jam-resources/python/examples/Walking/Walking/inputs/comak_settings.xml";


# results_basename = 'walking'
ik_result_dir = './results/comak-inverse-kinematics'
# comak_result_dir = './results/comak'
jnt_mech_result_dir = './results/joint-mechanics'

os.makedirs('./inputs', exist_ok=True)
os.makedirs('./results/graphics', exist_ok=True)

with open('./marker_weights.json', 'r') as f:
    marker_weights = json.load(f)

with open('./prescribed_coordinates.json', 'r') as f:
    prescribed_coordinates = json.load(f)

with open('./primary_coordinates.json', 'r') as f:
    primary_coordinates = json.load(f)

with open('./secondary_coordinates.json', 'r') as f:
    secondary_coordinates = json.load(f)

### comak_ik setting

In [3]:
# Settings
perform_secondary_constraint_sim = True
secondary_constraint_sim_settle_threshold = 1e-4
secondary_constraint_sim_sweep_time = 3.0
secondary_coupled_coordinate_start_value = 0
secondary_coupled_coordinate_stop_value = 100
secondary_constraint_sim_integrator_accuracy = 1e-2
secondary_constraint_sim_internal_step_limit = 10000
constraint_function_num_interpolation_points = 20
print_secondary_constraint_sim_results = True
perform_inverse_kinematics = True

report_errors = True
report_marker_locations = False
ik_constraint_weight = 100
ik_accuracy = 1e-5
use_visualizer = False

In [4]:
comak_ik = osim.COMAKInverseKinematicsTool();
comak_ik.set_model_file(model_file);
comak_ik.set_results_directory(ik_result_dir);
comak_ik.set_results_prefix(results_basename);
comak_ik.set_perform_secondary_constraint_sim(perform_secondary_constraint_sim);
comak_ik.set_secondary_coordinates(0,'/jointset/knee_r/knee_add_r');
comak_ik.set_secondary_coordinates(1,'/jointset/knee_r/knee_rot_r');
comak_ik.set_secondary_coordinates(2,'/jointset/knee_r/knee_tx_r');
comak_ik.set_secondary_coordinates(3,'/jointset/knee_r/knee_ty_r');
comak_ik.set_secondary_coordinates(4,'/jointset/knee_r/knee_tz_r');
comak_ik.set_secondary_coordinates(5,'/jointset/pf_r/pf_flex_r');
comak_ik.set_secondary_coordinates(6,'/jointset/pf_r/pf_rot_r');
comak_ik.set_secondary_coordinates(7,'/jointset/pf_r/pf_tilt_r');
comak_ik.set_secondary_coordinates(8,'/jointset/pf_r/pf_tx_r');
comak_ik.set_secondary_coordinates(9,'/jointset/pf_r/pf_ty_r');
comak_ik.set_secondary_coordinates(10,'/jointset/pf_r/pf_tz_r');
comak_ik.set_secondary_coupled_coordinate('/jointset/knee_r/knee_flex_r');
comak_ik.set_secondary_constraint_sim_settle_threshold(secondary_constraint_sim_settle_threshold);
comak_ik.set_secondary_constraint_sim_sweep_time(secondary_constraint_sim_sweep_time);
comak_ik.set_secondary_coupled_coordinate_start_value(secondary_coupled_coordinate_start_value);
comak_ik.set_secondary_coupled_coordinate_stop_value(secondary_coupled_coordinate_stop_value);
comak_ik.set_secondary_constraint_sim_integrator_accuracy(secondary_constraint_sim_integrator_accuracy);
comak_ik.set_secondary_constraint_sim_internal_step_limit(secondary_constraint_sim_internal_step_limit);
comak_ik.set_secondary_constraint_function_file(
    './results/comak-inverse-kinematics/secondary_coordinate_constraint_functions.xml');
comak_ik.set_constraint_function_num_interpolation_points(constraint_function_num_interpolation_points);
comak_ik.set_print_secondary_constraint_sim_results(print_secondary_constraint_sim_results);
comak_ik.set_constrained_model_file('./results/comak-inverse-kinematics/ik_constrained_model.osim');
comak_ik.set_perform_inverse_kinematics(perform_inverse_kinematics);
comak_ik.set_marker_file(os.path.join(models_path, 'knee_healthy/experimental_data/motion_analysis/overground_17.trc'));

comak_ik.set_output_motion_file('overground_17_ik.mot');
comak_ik.set_time_range(0, 0);
comak_ik.set_time_range(1, 2.36);
comak_ik.set_report_errors(report_errors);
comak_ik.set_report_marker_locations(report_marker_locations);
comak_ik.set_ik_constraint_weight(ik_constraint_weight);
comak_ik.set_ik_accuracy(ik_accuracy);
comak_ik.set_use_visualizer(use_visualizer);
# comak_ik.set_verbose(10);

In [5]:
ik_task_set = comak_ik.get_IKTaskSet()

In [6]:
ik_task_set = osim.IKTaskSet()
ik_task = osim.IKMarkerTask()

for marker, weight in marker_weights.items():
    ik_task.setName(marker)
    ik_task.setWeight(weight)
    ik_task_set.cloneAndAppend(ik_task)

In [7]:
comak_ik.set_IKTaskSet(ik_task_set)
comak_ik.printToXML('./inputs/comak_inverse_kinematics_settings.xml')

print('Running COMAKInverseKinematicsTool...')
comak_ik.run();

Running COMAKInverseKinematicsTool...
[critical] 
[critical] ==========================
[critical] COMAKInverseKinematicsTool
[critical] ==========================
[critical] 
[info] Loaded model lenhart2015 from file /root/jam-resources/models/knee_healthy/lenhart2015/lenhart2015.osim
[info] Secondary Coordinates:
[info] ----------------------
[info] knee_add_r
[info] knee_rot_r
[info] knee_tx_r
[info] knee_ty_r
[info] knee_tz_r
[info] pf_flex_r
[info] pf_rot_r
[info] pf_tilt_r
[info] pf_tx_r
[info] pf_ty_r
[info] pf_tz_r
[info] Secondary Coupled Coordinate: 
[info] Settle Threshold: 0.000100000
[info] Sweep Time: 3.0
[info] Sweep secondary_coupled_coordinate start value: 0.0
[info] Sweep secondary_coupled_coordinate stop value: 100.0
[info] Performing IK Secondary Constraint Simulation...
[info] Starting Settling Simulation.
[info] Time: 0.01
[info] Name                           Value                Value Change        
[info] knee_add_r                     -0.00184684          0.0 

### Setup COMAKTool

In [8]:
comak = osim.COMAKTool();
comak.set_model_file(model_file);
comak.set_coordinates_file(coordinates_file);
comak.set_external_loads_file(external_loads_file)
comak.set_results_directory(comak_result_dir);
comak.set_results_prefix(results_basename);
comak.set_replace_force_set(False);
comak.set_force_set_file(forceset_file));
comak.set_start_time(1.16);
comak.set_stop_time(2.32);
comak.set_time_step(0.01);
comak.set_lowpass_filter_frequency(6);
comak.set_print_processed_input_kinematics(True);

for coordinate_number, path in prescribed_coordinates.items():
    comak.set_prescribed_coordinates(int(coordinate_number), path)

for coord_number, path in primary_coordinates.items():
    comak.set_primary_coordinates(int(coord_number), path)

rist)
Frame 58 (t=0.29):	total squared error = 0.014421, marker error: RMS=0.0192294, max=0.0485138 (L.Wrist)
Frame 59 (t=0.295):	total squared error = 0.0157786, marker error: RMS=0.0201141, max=0.0500947 (L.Wrist)
Frame 60 (t=0.3):	total squared error = 0.0144924, marker error: RMS=0.0192769, max=0.049095 (L.Wrist)
Frame 61 (t=0.305):	total squared error = 0.0152092, marker error: RMS=0.0197479, max=0.0495633 (L.Wrist)
Frame 62 (t=0.31):	total squared error = 0.0143865, marker error: RMS=0.0192063, max=0.0462431 (L.Wrist)
Frame 63 (t=0.315):	total squared error = 0.0148388, marker error: RMS=0.019506, max=0.0461179 (L.Wrist)
Frame 64 (t=0.32):	total squared error = 0.0153153, marker error: RMS=0.0198167, max=0.0420126 (L.Wrist)
Frame 65 (t=0.325):	total squared error = 0.0164748, marker error: RMS=0.0205531, max=0.0437852 (L.Wrist)
Frame 66 (t=0.33):	total squared error = 0.0138224, marker error: RMS=0.018826, max=0.0393298 (L.Wrist)
Frame 67 (t=0.335):	total squared error = 0.014306

SyntaxError: invalid syntax (2091335479.py, line 8)

0.00826639, marker error: RMS=0.0145588, max=0.0344481 (R.Bicep)
Frame 191 (t=0.955):	total squared error = 0.00822895, marker error: RMS=0.0145258, max=0.0340729 (R.Bicep)
Frame 192 (t=0.96):	total squared error = 0.00818465, marker error: RMS=0.0144866, max=0.0353601 (R.Bicep)
Frame 193 (t=0.965):	total squared error = 0.00828951, marker error: RMS=0.0145792, max=0.0343284 (R.Bicep)
Frame 194 (t=0.97):	total squared error = 0.00828484, marker error: RMS=0.014575, max=0.0355192 (R.Bicep)
Frame 195 (t=0.975):	total squared error = 0.00839495, marker error: RMS=0.0146716, max=0.0344114 (R.Bicep)
Frame 196 (t=0.98):	total squared error = 0.0091518, marker error: RMS=0.0153187, max=0.0339403 (R.Bicep)
Frame 197 (t=0.985):	total squared error = 0.00818325, marker error: RMS=0.0144854, max=0.0341995 (R.Bicep)
Frame 198 (t=0.99):	total squared error = 0.00825876, marker error: RMS=0.0145521, max=0.0344644 (R.Bicep)
Frame 199 (t=0.995):	total squared error = 0.00878727, marker error: RMS=0.01

In [ ]:
print(coordinate_number)

In [ ]:
secondary_coord_set = osim.COMAKSecondaryCoordinateSet(); 
secondary_coord = osim.COMAKSecondaryCoordinate();

for coord, dict_ in secondary_coordinates.items():
    secondary_coord.setName(coord)
    secondary_coord.set_max_change(dict_['max_change']);
    secondary_coord.set_coordinate(dict_['coordinate']);
    secondary_coord_set.cloneAndAppend(secondary_coord);

comak.set_COMAKSecondaryCoordinateSet(secondary_coord_set);

In [ ]:
comak.set_settle_secondary_coordinates_at_start(True);
comak.set_settle_threshold(1e-3);
comak.set_settle_accuracy(1e-2);
comak.set_settle_internal_step_limit(10000);
comak.set_print_settle_sim_results(True);
comak.set_settle_sim_results_directory(comak_result_dir);
comak.set_settle_sim_results_prefix('walking_settle_sim');
comak.set_max_iterations(25);
comak.set_udot_tolerance(1);
comak.set_udot_worse_case_tolerance(50);
comak.set_unit_udot_epsilon(1e-6);
comak.set_optimization_scale_delta_coord(1);
comak.set_ipopt_diagnostics_level(3);
comak.set_ipopt_max_iterations(500);
comak.set_ipopt_convergence_tolerance(1e-4);
comak.set_ipopt_constraint_tolerance(1e-4);
comak.set_ipopt_limited_memory_history(200);
comak.set_ipopt_nlp_scaling_max_gradient(10000);
comak.set_ipopt_nlp_scaling_min_value(1e-8);
comak.set_ipopt_obj_scaling_factor(1);
comak.set_activation_exponent(2);
comak.set_contact_energy_weight(0);
comak.set_non_muscle_actuator_weight(1000);
comak.set_model_assembly_accuracy(1e-12);
comak.set_use_visualizer(False);
# # comak.set_verbose(2);
comak.setDebugLevel(2)
comak.printToXML(save_xml_path);



In [ ]:
print('Running COMAK Tool...')
comak.run();
print('Done!')

### Perform Joint Mechanics Analysis

In [ ]:
## Perform Joint Mechanics Analysis
jnt_mech = osim.JointMechanicsTool();
jnt_mech.set_model_file(model_file);
jnt_mech.set_input_states_file([os.path.join(comak_result_dir, results_basename + '_states.sto')]);
jnt_mech.set_use_muscle_physiology(False);
jnt_mech.set_results_file_basename(results_basename);
jnt_mech.set_results_directory(jnt_mech_result_dir);
jnt_mech.set_start_time(1.16);
jnt_mech.set_stop_time(-1);
jnt_mech.set_resample_step_size(-1);
jnt_mech.set_normalize_to_cycle(True);
jnt_mech.set_lowpass_filter_frequency(-1);
jnt_mech.set_print_processed_kinematics(False);
jnt_mech.set_contacts(0,'all');
jnt_mech.set_contact_outputs(0,'all');
jnt_mech.set_contact_mesh_properties(0,'none');
jnt_mech.set_ligaments(0,'all');
jnt_mech.set_ligament_outputs(0,'all');
jnt_mech.set_muscles(0,'all');
jnt_mech.set_muscle_outputs(0,'all');

jnt_mech.set_attached_geometry_bodies(0,'all');

jnt_mech.set_output_orientation_frame('ground');
jnt_mech.set_output_position_frame('ground');
jnt_mech.set_write_vtp_files(True);
jnt_mech.set_vtp_file_format('binary');
jnt_mech.set_write_h5_file(False);
jnt_mech.set_h5_kinematics_data(True);
jnt_mech.set_h5_states_data(True);
jnt_mech.set_write_transforms_file(False);
jnt_mech.set_output_transforms_file_type('sto');
jnt_mech.set_use_visualizer(False);
jnt_mech.set_verbose(0);

analysis_set = AnalysisSet();

frc_reporter = ForceReporter();
frc_reporter.setName('ForceReporter');

analysis_set.cloneAndAppend(frc_reporter);
jnt_mech.set_AnalysisSet(analysis_set);
jnt_mech.printToXML('./inputs/joint_mechanics_settings.xml');

print('Running JointMechanicsTool...');
jnt_mech.run();

r error: RMS=0.0170615, max=0.0333304 (L.TH3)
Frame 323 (t=1.615):	total squared error = 0.0114566, marker error: RMS=0.0171394, max=0.0350878 (L.TH3)
Frame 324 (t=1.62):	total squared error = 0.011839, marker error: RMS=0.0174231, max=0.0353306 (L.TH3)
Frame 325 (t=1.625):	total squared error = 0.0119745, marker error: RMS=0.0175225, max=0.0368349 (L.TH3)
Frame 326 (t=1.63):	total squared error = 0.0118209, marker error: RMS=0.0174098, max=0.0368162 (L.TH3)
Frame 327 (t=1.635):	total squared error = 0.0121648, marker error: RMS=0.0176612, max=0.0353059 (L.TH3)
Frame 328 (t=1.64):	total squared error = 0.0126148, marker error: RMS=0.0179849, max=0.0365106 (L.TH3)
Frame 329 (t=1.645):	total squared error = 0.0122593, marker error: RMS=0.0177297, max=0.0368295 (L.TH3)
Frame 330 (t=1.65):	total squared error = 0.012742, marker error: RMS=0.0180753, max=0.0352115 (L.TH3)
Frame 331 (t=1.655):	total squared error = 0.0137974, marker error: RMS=0.018809, max=0.0333203 (L.TH3)
Frame 332 (t=1.6